## Preambule

In [1]:
import numpy as np
from tqdm import tqdm
from importlib import reload
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import scipy
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Run classes

Data reader class

In [2]:
import class_datareading
reload(class_datareading)
from class_datareading import datareading

datareader = datareading()
datareader.read_general()
datareader.read_ssps()
datareader.read_undata()
datareader.read_hdi()
datareader.read_historicalemis_jones()
datareader.read_ar6()
datareader.relation_budget_nonco2()
datareader.determine_global_nonco2_trajectories()
datareader.determine_global_budgets()
datareader.determine_global_co2_trajectories()
datareader.read_baseline()
datareader.read_ndc()
datareader.merge_xr()
datareader.add_country_groups()
datareader.save()
datareader.country_specific_datareaders()

# ==================================== #
# DATAREADING class                    #
# startyear:  2021
# ==================================== #
- Reading general data
- Reading GDP and population data from SSPs
- Reading UN population data and gapminder, processed by OWID (for past population)
- Read Human Development Index data
- Reading historical emissions (jones)
- Read AR6 data
- Get relationship between CO2 budgets and non-co2 reduction in 2050
- Computing global nonco2 trajectories
- Get global CO2 budgets
- Computing global co2 trajectories
- Reading baseline emissions
- Reading NDC data
- Merging xrarray object
- Add country groups
- Save important files


In [3]:
import class_allocation
reload(class_allocation)
from class_allocation import allocation

allocator = allocation('USA', lulucf='excl', gas='CO2')
allocator.gf()
allocator.pc()
allocator.pcc()
allocator.pcb()


In [4]:
self = allocator

In [5]:
# Step 1: Reductions before correction factor
xrt = self.xr_total.sel(Time=self.analysis_timeframe)
GDP_sum_w = xrt.GDP.sel(Region='EARTH')
pop_sum_w = xrt.Population.sel(Region='EARTH')
# Global average GDP per capita
r1_nom = GDP_sum_w / pop_sum_w

base_worldsum = self.emis_base.sel(Time=self.analysis_timeframe).sel(Region='EARTH')
rb_part1 = (xrt.GDP.sel(Region=self.focus_region) / xrt.Population.sel(Region=self.focus_region) / r1_nom)**(1/3.)
rb_part2 = self.emis_base.sel(Time=self.analysis_timeframe).sel(Region=self.focus_region) * (base_worldsum - self.emis_fut.sel(Time=self.analysis_timeframe)) / base_worldsum
rb = rb_part1 * rb_part2

# Step 2: Correction factor
corr_factor = (1e-9+self.rbw.__xarray_dataarray_variable__)/(base_worldsum - self.emis_fut.sel(Time=self.analysis_timeframe))

# Step 3: Budget after correction factor
ap = self.emis_base.sel(Region=self.focus_region) - rb/corr_factor

ap = ap.sel(Time=self.analysis_timeframe)
#self.xr_total = self.xr_total.assign(AP = ap)
#self.rbw.close()


In [6]:
# make sqrt curve
compensation_form_sqrt = np.sqrt(np.arange(0, 2101 - self.start_year_analysis))
# sum of values has to be 1
compensation_form_sqrt = compensation_form_sqrt / np.sum(compensation_form_sqrt)

xr_comp = xr.DataArray(compensation_form_sqrt, dims=['Time'], coords={'Time': self.analysis_timeframe})

# Defining the timeframes for historical and future emissions
xrs = []
hist_emissions_startyears = self.settings['params']['hist_emissions_startyears']
discount_rates = self.settings['params']['discount_rates']

for startyear in hist_emissions_startyears:
    hist_emissions_timeframe = np.arange(startyear, 1 + self.start_year_analysis)
    future_emissions_timeframe = np.arange(self.start_year_analysis + 1, 2101)

    # Summing all historical CO2 emissions over the hist_emissions_timeframe
    hist_emissions = self.emis_hist.sel(Time = hist_emissions_timeframe)

    # Discounting -> We only do past discounting here
    for discount in discount_rates:
        past_timeline = np.arange(startyear, self.start_year_analysis + 1)

        discount_factor = (1 - discount / 100)
        discount_period = self.start_year_analysis - past_timeline
        xr_discount = xr.DataArray(discount_factor ** discount_period, dims=['Time'],
                                coords={'Time': past_timeline})
        hist_emissions_discounted = (hist_emissions * xr_discount).sum(dim='Time')
        hist_emissions_w = float(hist_emissions_discounted.sel(Region='EARTH'))
        hist_emissions_r = float(hist_emissions_discounted.sel(Region = self.focus_region))

        # Summing all future emissions over the future_emissions_timeframe
        future_emissions_w = self.emis_fut.sel(Time = future_emissions_timeframe).sum(dim='Time')

        total_emissions_w = hist_emissions_w + future_emissions_w

        # Calculating the cumulative population shares for region and world
        cum_pop = self.xr_total.Population.sel(Time = self.analysis_timeframe).sum(dim='Time')
        cum_pop_r = cum_pop.sel(Region=self.focus_region)
        cum_pop_w = cum_pop.sel(Region='EARTH')
        share_cum_pop = cum_pop_r / cum_pop_w
        budget_rightful = total_emissions_w * share_cum_pop
        budget_left = budget_rightful - hist_emissions_r

        # Now temporal allocation
        #globalbudget = self.xr_total.CO2_globe.sel(Time=self.analysis_timeframe).sum(dim='Time')
        globalpath = self.emis_fut

        emis_start_r = self.emis_hist.sel(Time=self.start_year_analysis,
                                                    Region=self.focus_region)
        emis_start_w = self.emis_hist.sel(Time=self.start_year_analysis,
                                                    Region='EARTH')
        emis_ratio = emis_start_r / emis_start_w
        path_scaled_0 = emis_ratio * globalpath
        budget_without_assumptions = path_scaled_0.sum(dim='Time')
        budget_surplus = budget_left - budget_without_assumptions

        def ecpc_factor(initial_path, f):
            '''
            Calculates a modified emissions path. Takes scaling factor f as input and
            returns a new emissions path by scaling the compensation form (xr_comp) with f
            and adding it to the initial emissions path (path_scaled_0).
            '''
            return initial_path + xr_comp * f

        ecpc = ecpc_factor(path_scaled_0, budget_surplus)
        ecpc_expanded = ecpc.expand_dims(Discount_factor=[discount],
                                            Historical_startyear=[startyear]).to_dataset(name='ECPC')
        xrs.append(ecpc_expanded)

xr_ecpc = xr.merge(xrs)

In [21]:
path_scaled_0.sum(dim='Time').sel(NegEmis=0.5, NonCO2red=0.5, Temperature=1.6, Risk=0.5, Timing='Immediate')
(ap.sel(Time=np.arange(self.settings['params']['start_year_analysis'], 2101))*xr.where(self.emis_fut.sel(Time=np.arange(self.settings['params']['start_year_analysis'], 2101)) > 0, 1, 0))

<xarray.DataArray ()> Size: 8B
array(34594.35997645)
Coordinates:
    Variable     <U24 96B 'Emissions|N2O|AFOLU|Land'
    Temperature  float64 8B 1.6
    Risk         float64 8B 0.5
    NonCO2red    float64 8B 0.5
    NegEmis      float64 8B 0.5
    Timing       <U9 36B 'Immediate'

In [22]:
path_scaled_0.sel(NegEmis=0.5, NonCO2red=0.5, Temperature=1.6, Risk=0.5, Timing='Immediate')

<xarray.DataArray (Time: 251)> Size: 2kB
array([          nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
                 nan,           nan,           nan,           nan,
...
       4740.85615611, 4442.53538532, 4144.21461452, 3845.89384373,
       3547.57307294, 3249.25230215, 2950.93153135, 2667.42410604,
       2437.83144235, 2259.97409749, 2116.0911072 , 1972.20811691,
       1828.32512663, 1684.44213634, 1540.55914605, 1396.67615576,
       1252.79316547, 1108.91017518,  989.93228653,  885.47574996,
        805.92431501,  726.37288007,  646.82144513,  567.27001019,
        487.71857525,  408.16714031,  328.61570537,  249.06427043,
        185.64622799,  129.19216127,   88.87148704,   48.55081282,
          8.23013859,  -32.09053563,  -72.41120986, -112.73188408,
       -153.0525583 , -193.37323253, -224.95199675, -258.50420726,
       -283.31450777, -308.12480829, -332.9351088 , -357.74540931,
       -382.55570982, -407.36601033, -432.17631084, -456.98661135,
       -479.06203136, -495.89685545, -509.99679904, -524.09674263,
       -538.19668622, -552.29662981, -566.3965734 , -580.49651699,
       -594.59646058, -608.69640417, -621.34369694, -630.7498007 ,
       -638.70325363, -646.65670656, -654.6101595 , -662.56361243,
       -670.51706537, -678.4705183 , -686.42397124, -694.37742417,
       -701.35108221, -709.96785847, -717.60483984, -725.24182121,
       -732.87880258, -740.51578395, -748.15276532, -755.78974669,
       -763.42672806, -771.06370943, -778.7006908 ])
Coordinates:
  * Time         (Time) int32 1kB 1850 1851 1852 1853 ... 2097 2098 2099 2100
    Variable     <U24 96B 'Emissions|N2O|AFOLU|Land'
    Temperature  float64 8B 1.6
    Risk         float64 8B 0.5
    NonCO2red    float64 8B 0.5
    NegEmis      float64 8B 0.5
    Timing       <U9 36B 'Immediate'

In [19]:
budget_left.sel(NegEmis=0.5, NonCO2red=0.5, Temperature=1.6, Risk=0.5, Scenario='SSP2', Timing='Immediate')

<xarray.DataArray ()> Size: 8B
array(-83702.52432725)
Coordinates:
    Temperature  float64 8B 1.6
    Risk         float64 8B 0.5
    NonCO2red    float64 8B 0.5
    NegEmis      float64 8B 0.5
    Timing       <U9 36B 'Immediate'
    Variable     <U24 96B 'Emissions|N2O|AFOLU|Land'
    Scenario     <U4 16B 'SSP2'

In [17]:
budget_without_assumptions.sel(NegEmis=0.5, NonCO2red=0.5, Temperature=1.6, Risk=0.5, Timing='Immediate')

<xarray.DataArray ()> Size: 8B
array(34594.35997645)
Coordinates:
    Variable     <U24 96B 'Emissions|N2O|AFOLU|Land'
    Temperature  float64 8B 1.6
    Risk         float64 8B 0.5
    NonCO2red    float64 8B 0.5
    NegEmis      float64 8B 0.5
    Timing       <U9 36B 'Immediate'

In [13]:
ecpc.sum(dim='Time').sel(NegEmis=0.5, NonCO2red=0.5, Temperature=1.6, Risk=0.5, Scenario='SSP2', Timing='Immediate')

<xarray.DataArray ()> Size: 8B
array(-83702.52432725)
Coordinates:
    Temperature  float64 8B 1.6
    Risk         float64 8B 0.5
    NonCO2red    float64 8B 0.5
    NegEmis      float64 8B 0.5
    Timing       <U9 36B 'Immediate'
    Variable     <U24 96B 'Emissions|N2O|AFOLU|Land'
    Scenario     <U4 16B 'SSP2'

In [7]:
import class_allocation
reload(class_allocation)
from class_allocation import allocation

regions_iso = np.load("K:/Data/Data_effortsharing/DataUpdate_ongoing/all_regions.npy", allow_pickle=True)
for cty in tqdm(regions_iso[122:]):
    allocator = allocation(cty, lulucf='excl', gas='CO2')
    allocator.gf()
    allocator.pc()
    allocator.pcc()
    allocator.pcb()
    allocator.ecpc()
    allocator.ap()
    allocator.gdr()
    allocator.save()

 56%|█████▋    | 48/85 [32:19<24:54, 40.40s/it]


KeyboardInterrupt: 

Allocations

In [4]:
import class_allocation
reload(class_allocation)
from class_allocation import allocation

for lulucf in ['incl', 'excl']:
    for gas in ['GHG', 'CO2']:
        for dataread_file in ['xr_dataread.nc', 'xr_dataread_nor.nc']: # This is only necessary if for a country a specific historical emissions profile is required beyond what is in Jones (e.g. Norway was requested)
            allocator = allocation('NOR', lulucf=lulucf, gas=gas, dataread_file=dataread_file)
            allocator.gf()
            allocator.pc()
            allocator.pcc()
            allocator.pcb()
            allocator.ecpc()
            allocator.ap()
            allocator.gdr()
            allocator.save()
            print('NOR', lulucf, gas, dataread_file, 'done')

NOR incl GHG xr_dataread.nc done
NOR incl GHG xr_dataread_nor.nc done
NOR incl CO2 xr_dataread.nc done
NOR incl CO2 xr_dataread_nor.nc done
NOR excl GHG xr_dataread.nc done
NOR excl GHG xr_dataread_nor.nc done
NOR excl CO2 xr_dataread.nc done
NOR excl CO2 xr_dataread_nor.nc done


Sanity check CSV construction

In [68]:
xr_primap2 = xr.open_dataset("X:/user/dekkerm/Data/PRIMAP/Guetschow_et_al_2024-PRIMAP-hist_v2.5.1_final_no_rounding_27-Feb-2024.nc")
xr_primap_lu = xr_primap2['KYOTOGHG (AR6GWP100)'].rename({'area (ISO3)': 'Region', 'scenario (PRIMAP-hist)': 'scen', 'category (IPCC2006_PRIMAP)': 'cat'}).sel(scen='HISTTP', provenance='derived', cat=['M.LULUCF'], source='PRIMAP-hist_v2.5.1_final_nr').sum(dim='cat').drop_vars(['source', 'provenance', 'scen'])
xr_primap_lu['time'] = np.arange(1750, 2023)
xr_primap_lu = xr_primap_lu.rename({'time': 'Time'})

incl = datareader.xr_total.GHG_hist
excl = datareader.xr_total.GHG_hist_excl
afolu_jones = (datareader.xr_ghg_afolu*1e3).rename({'Data': 'GHG_AFOLU_Jones'})
agriculture_primap = datareader.xr_total.assign(GHG_AGRI_PRIMAP = datareader.xr_ghg_agri*1e3).GHG_AGRI_PRIMAP
diff = datareader.xr_total.assign(GHG_LULUCF_subtracted = incl - excl).GHG_LULUCF_subtracted
lulucf_primap = datareader.xr_total.assign(GHG_LULUCF_PRIMAP = xr_primap_lu/1e3).GHG_LULUCF_PRIMAP

xr.merge([incl, excl, afolu_jones, agriculture_primap, diff, lulucf_primap]).sel(Region=['AUS', 'CHN', 'CAN', 'EU', 'GBR', 'ARG', 'JPN', 'IND', 'BRA', 'RUS', 'IDN', 'ZAF', 'USA', 'MEX', 'TUR', 'KOR', 'SAU', 'G20', 'EARTH', 'NLD'], Time=[2015, 2021]).drop_vars(['Variable', 'variable']).to_dataframe().unstack().to_csv('C:/Users/dekkerm\OneDrive - Planbureau voor de Leefomgeving/Documenten/Other/GHG_hist.csv')

<>:8: SyntaxWarning: invalid escape sequence '\O'
<>:8: SyntaxWarning: invalid escape sequence '\O'
C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_4248\4075408136.py:8: SyntaxWarning: invalid escape sequence '\O'
  xr.merge([incl, excl, afolu_jones, agriculture_primap, diff, lulucf_primap]).sel(Region=['AUS', 'CHN', 'CAN', 'EU', 'GBR', 'ARG', 'JPN', 'IND', 'BRA', 'RUS', 'IDN', 'ZAF', 'USA', 'MEX', 'TUR', 'KOR', 'SAU', 'G20', 'EARTH', 'NLD'], Time=[2015, 2021]).drop_vars(['Variable', 'variable']).to_dataframe().unstack().to_csv('C:/Users/dekkerm\OneDrive - Planbureau voor de Leefomgeving/Documenten/Other/GHG_hist.csv')


Allocation class

In [63]:
import class_allocation
reload(class_allocation)
from class_allocation import allocation

for cty in ['USA', 'EU', 'BRA', 'EARTH']:#tqdm(np.array(datareader.xr_total.Region)):
    allocator = allocation(cty)
    allocator.gf()
    allocator.pc()
    allocator.pcc()
    allocator.pcb()
    allocator.ecpc()
    allocator.ap()
    allocator.gdr()
    allocator.save()

Allocation rules combining approaches (Robiou paper)

In [2]:
import class_allocation_combinedapproaches
reload(class_allocation_combinedapproaches)
from class_allocation_combinedapproaches import allocation_comb

allocator = allocation_comb(lulucf='excl', gas='GHG')
allocator.ecpc()
allocator.discounting_historical_emissions()
allocator.approach1gdp()
allocator.approach1hdi()
allocator.approach2()
allocator.approach2_transition()
allocator.combine()
allocator.get_relation_2030emis_temp()
allocator.determine_tempoutcomes()
allocator.save()

Temperature NDC-alignment metric

In [3]:
import class_tempalign
reload(class_tempalign)
from class_tempalign import tempaligning

tempaligner = tempaligning() # FIRST RUN AGGREGATOR FOR THIS!! (2030 alloc)
tempaligner.get_relation_2030emis_temp()
tempaligner.determine_tempoutcomes()
tempaligner.save()

# ==================================== #
# Initializing tempaligning class        #
# ==================================== #
- Determine relation between 2030-emissions and temperature outcome
- Determine temperature metric


100%|██████████| 6/6 [00:11<00:00,  1.96s/it]


- Save


Reading policy scenarios from ENGAGE

In [2]:
import class_policyscens
reload(class_policyscens)
from class_policyscens import policyscenadding

policyscenner = policyscenadding()
policyscenner.read_engage_data()
policyscenner.filter_and_convert()
policyscenner.add_to_xr()

# ==================================== #
# Initializing policyscenadding class  #
# ==================================== #
- Read ENGAGE scenarios and change region namings
- Filter correct scenarios and convert to xarray object
- Add to overall xrobject


Variance decomposition

In [5]:
import class_variancedecomp
reload(class_variancedecomp)
from class_variancedecomp import vardecomposing

vardecomposer = vardecomposing()
vardecomposer.sobolindices = {}
print('- Starting sobols for fixed years, over many countries')
timeseries = np.arange(2030, 2101, 5)
for year in timeseries:
    print('  Starting with', year)
    xr_cty, ar_time, array_dims, array_inputs, problem, samples = vardecomposer.prepare_global_sobol(year)
    vardecomposer.sobolindices[year] = vardecomposer.apply_decomposition(xr_cty, ar_time, array_dims, array_inputs, problem, samples)
vardecomposer.save(array_dims, timeseries)

# ==================================== #
# Initializing vardecomposing class    #
# ==================================== #
- Starting sobols for fixed years, over many countries
  Starting with 2030


207it [02:13,  1.55it/s]


  Starting with 2035


207it [02:24,  1.43it/s]


  Starting with 2040


...

207it [02:27,  1.41it/s]


  Starting with 2100


207it [02:24,  1.43it/s]


- Save global results


FileNotFoundError: [Errno 2] No such file or directory: 'K:\\data\\DataUpdate_ongoing\\xr_alloc_2030.nc'